In [2]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 5.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78935 sha256=9e060a4cf88550451a4313a3d8e84f54aa44f77c56f449905e835dcc5578738c
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=dac6373a193ebe9f699f7f3569cc2f6c701a9bf0520ea9cc20b15f89f51d0b93
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
train_images = train_images/255.0
test_images = test_images/255.0

In [16]:
train_images.shape

(60000, 28, 28, 1)

In [8]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [9]:
def built_model(hp):
  model = keras.Sequential([keras.layers.Conv2D(
      filters = hp.Int('conv_1_filter', min_value = 32, max_value = 128, step =16),
      kernel_size = hp.Choice('conv_1_kernel', values =[3,5]),
      activation = 'relu',
      input_shape = (28,28,1)
  ),
  keras.layers.Conv2D(
      filters = hp.Int('conv_2_filter', min_value = 32, max_value = 64, step =16),
      kernel_size = hp.Choice('conv_2_kernel', values =[3,5]),
      activation = 'relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units = hp.Int('dense_1_units', min_value = 32, max_value = 128, step = 16),
          activation = 'relu'
      ),
      keras.layers.Dense(10, activation = 'softmax')
  ])

  model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', values = [1e-2, 1e-3])),
                loss= 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search = RandomSearch(built_model, objective= 'val_accuracy', max_trials = 5, directory = 'output', project_name="Mnist Fashion")

In [12]:
tuner_search.search(train_images, train_labels, epochs = 3, validation_split = 0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.8638333082199097

Best val_accuracy So Far: 0.9085000157356262
Total elapsed time: 00h 04m 24s
INFO:tensorflow:Oracle triggered exit


In [13]:
model = tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        76848     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               2973824   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 3,052,602
Trainable params: 3,052,602
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(train_images, train_labels, epochs =10, validation_split =0.1, initial_epoch = 3)

Epoch 4/10
1688/1688 [==============================] - 15s 8ms/step - loss: 0.1288 - accuracy: 0.9529 - val_loss: 0.2502 - val_accuracy: 0.9143
Epoch 5/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0889 - accuracy: 0.9670 - val_loss: 0.3031 - val_accuracy: 0.9073
Epoch 6/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0629 - accuracy: 0.9769 - val_loss: 0.3397 - val_accuracy: 0.9132
Epoch 7/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0442 - accuracy: 0.9841 - val_loss: 0.4175 - val_accuracy: 0.9135
Epoch 8/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0339 - accuracy: 0.9881 - val_loss: 0.4613 - val_accuracy: 0.9150
Epoch 9/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0303 - accuracy: 0.9894 - val_loss: 0.4403 - val_accuracy: 0.9097
Epoch 10/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0232 - accuracy: 0.9917 - val_loss: 0.5692 - val_a

In [18]:
pred = model.predict_classes(test_images)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [20]:
pred[0:10]

array([9, 2, 1, 1, 6, 1, 4, 6, 5, 7])

In [22]:
test_labels[0:10]

array([9, 2, 1, 1, 6, 1, 4, 6, 5, 7], dtype=uint8)